In [1]:
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from pydantic import BaseModel, Field

load_dotenv()

/home/sonsungjong/python/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
# Pydantic : 데이터 유효성 검사
# get_format_instructions() : 원하는 포맷 지정
# parse() : 객체로 반환
# llm = ChatOpenAI(model="gpt-5-nano")
# llm = ChatOllama(model="gemma3:1b", base_url="http://localhost:11434")


In [3]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [4]:
prompt = PromptTemplate.from_template(f"다음의 이메일 내용 중 중요한 내용을 추출해 주세요.\n\n{email_conversation}")

llm = ChatOpenAI(model="gpt-5-nano", temperature=0)
# llm = ChatOllama(model="gemma3:1b", base_url="http://localhost:11434", temperature=0)

chain = prompt | llm

answer = ""
for chunk in chain.stream({"email_conversation":email_conversation}):
    answer += str(chunk.content)
    print(chunk.content, end="", flush=True)

print(f"\n\n[FINAL]{answer}")

다음은 이메일에서 추출한 중요한 내용입니다.

- 발신자/소속: 김철수 상무, 바이크코퍼레이션
- 수신자: 이은채 대리, teddy international
- 주제: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
- 주요 요청
  - ZENESIS 자전거의 상세 브로슈어 요청
  - 필요 항목: 기술 사양, 배터리 성능, 디자인 정보
  - 목적: 유통 전략 및 마케팅 계획을 보다 구체화하려는 것
- 미팅 제안
  - 일정: 다음 주 화요일(1월 15일) 오전 10시
  - 장소: 상대사 사무실에서 미팅 제안
  - 목적: 협력 가능성 논의
- 마무리 인사: 감사합니다. | 김철수, 상무이사, 바이크코퍼레이션

[FINAL]다음은 이메일에서 추출한 중요한 내용입니다.

- 발신자/소속: 김철수 상무, 바이크코퍼레이션
- 수신자: 이은채 대리, teddy international
- 주제: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
- 주요 요청
  - ZENESIS 자전거의 상세 브로슈어 요청
  - 필요 항목: 기술 사양, 배터리 성능, 디자인 정보
  - 목적: 유통 전략 및 마케팅 계획을 보다 구체화하려는 것
- 미팅 제안
  - 일정: 다음 주 화요일(1월 15일) 오전 10시
  - 장소: 상대사 사무실에서 미팅 제안
  - 목적: 협력 가능성 논의
- 마무리 인사: 감사합니다. | 김철수, 상무이사, 바이크코퍼레이션


In [12]:
# LLM의 답변 형식을 클래스로 정의
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요악한 텍스트")
    date: str = Field(description="메일 본문에 언급한 미팅 날짜와 시간")

parser = PydanticOutputParser(pydantic_object=EmailSummary)

prompt = PromptTemplate.from_template("""You are a helpful assistant.
Please answer the following questions in KOREAN.

#QUESTION:
{question}

#EMAIL_CONVERSATION:
{email_conversation}

FORMAT:
{format}
""")

prompt = prompt.partial(format=parser.get_format_instructions())

chain = prompt | llm | StrOutputParser()

answer = chain.invoke({"email_conversation":email_conversation, "question":"이메일 내용 중 주요 내용을 추출해 주세요."})
print(answer)


{
  "person": "김철수 상무",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "바이크코퍼레이션의 김철수 상무가 귀사의 신규 자전거 'ZENESIS'에 대해 관심을 표명하고, 기술 사양, 배터리 성능, 디자인 등 상세 브로슈어를 요청했습니다. 이를 바탕으로 자사 제안 유통 전략과 마케팅 계획을 구체화하려고 하며, 협력 가능성 논의를 위해 다음 주 화요일(1월 15일) 오전 10시 미팅을 제안했습니다.",
  "date": "다음 주 화요일(1월 15일) 오전 10시"
}


In [ ]:
llm = ChatOpenAI(model="gpt-5-nano", temperature=0).with_structured_output(EmailSummary)
answer = llm.invoke(email_conversation)
answer

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='RE: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='제안된 브로슈어 요청 및 미팅 일정에 대한 회신 초안 작성 요청에 대한 답변 초안입니다. 상대방이 보낸 브로슈어 요청과 다음 주 화요일 1월 15일 오전 10시의 미팅 제안에 대해 확인 및 준비 의사를 밝히고, 브로슈어 공유, 미팅 의제 및 필요 시 대면/화상 회의 옵션, NDA 등 협력 조건에 대한 논의 의향을 포함한 회신 초안입니다.', date='2025-09-22')